自编码器
===

# 1.概述
传统的机器学习任务很大程度上依赖于好的特征工程，但是特征工程往往是非常耗时耗力的，在图像、语音、视频中提取有效的特征就更难了。更关键的是，工程师必须在某种数据处理专业领域有很深的造诣，比如CV工程师，NLP工程师，语音处理工程师，量化交易工程师

# 2.无监督特征学习的特点
- 无监督学习：即我们不需要标注数据就可以对数据进行一定程度的学习，这种学习是对数据内容的组织形式的学习，提取的是频繁出现的特征
- 逐层抽象：特征是需要不断抽象的，就像人总是从简单基础的概念开始学习，在到复杂的概念

# 3.特征的粒度与稀疏编码
像素级别的特征没有任何意义，为了让机器可以从像素中提取出有效的特征，可以使用稀疏编码

## 3.1.稀疏编码
- 选择一组S[k]，然后调整a[k]，使得$\sum_k(a[k] * S[k])$最接近与T
- 固定住a[k]，在400个碎片中，选择其它更适合的碎片S'[k]，替换原先的S[k]，使得$\sum_k(a[k] * S'[k])$最接近T

经过几次迭代后，发现最佳的S[k]组合，基本是照片上不同物体的边缘线，这些线段形状相似，区别在于方向。这就说明，复杂图形，往往是有一些基本结构组成的，可以按照线段的不同权重调和而成。同样声音也是这样的。

## 3.2.Xaiver Initialization初始化
如果深度学习模型的权重初始化的太小，那信号将在每层间传递时逐渐缩小而难以产生作用，如果权重初始的太大，那信号将在每层间传递时逐渐放大并导致发散和失效，如果有了Xaiver初始化，它就是让权重满足0均值，同时方差为$\frac{1}{n_{in}+n_{out}}$，分布可以用均匀分布或高斯分布

# 4.AutoEncoder自动编码器
## 4.1.介绍
Deep Learning最简单一种方法就是利用人工神经网络的特点。人工神经网络ANN本身就是具有层次结构的系统，如果给定一个神经网络，我们假设输出与输入是相同的，然后训练调整其参数，得到每一层中的权重，自然的我们就得到了输入I的集中不同表示，其中没一层代表一种表示，这些表示就是特征，自动编码起就是一种尽可能浮现输入信号的神经网络。为了实现这种复现，自动编码起就必须捕捉可以代表输入数据的最重要的因素，就像PCA那样，找到可以代表原信息的主要成分。
- 输入一批无标签的数据，然后进行编码，紧接着进行解码，可以得到一批计算后的数据，然后跟原始数据进行比较，可以得到误差。我们需要做的就是调整编码和解码的参数，使得重构误差最小，这样就得到了特征
- 训练好一层之后，需要进行下一层的输入，同样假如一个编码器和一个解码器，然后固定第一层的编码器，同时去掉第一层的解码器，继续训练...同样的办法可以训练多层
- 这样就训练好了多层，最后我们需要假如一个分类器，比如SVM或者Logistic回归等，然后通过标准的多层神经网络的监督训练方法(梯度下降法)去训练
- 从输入到隐藏层，这一段神经元数量下降，被称为”encoder”，也可以称为模式识别模型或判别模型；而从隐藏层到输出，这一段神经元数量上升，被称为”decoder”，也称生成模型。因而编码器在一定程度上是类似于GAN
- 自编码器尝试使用一个函数，将输入逼近输出，经过训练之后，撤去输入层，仅凭隐藏层也可以实现重现输出
- 这样隐藏层就需要做到提取主要成分的任务。这样的功能实现了隐藏层对于数据特征的提取，类似于主成分分析

## 4.2.自编码器的一般结构
![images](../images/02_04_001.png)<br/>
从上面的图中，我们能够看到两个部分，第一个部分是编码器(Encoder)，第二个部分是解码器(Decoder)，编码器和解码器都可以是任意的模型，通常我们使用神经网络模型作为编码器和解码器。输入的数据经过神经网络降维到一个编码(code)，接着又通过另外一个神经网络去解码得到一个与输入原数据一模一样的生成数据，然后通过去比较这两个数据，最小化他们之间的差异来训练这个网络中编码器和解码器的参数。当这个过程训练完之后，我们可以拿出这个解码器，随机传入一个编码(code)，希望通过解码器能够生成一个和原数据差不多的数据，上面这种图这个例子就是希望能够生成一张差不多的图片。

# 5.常见自编码器
常见自编码器包括标准自编码器，变分自编码器，降噪自编码器，稀疏自编码器以及对抗自编码器

## 5.1.标准自编码器
标准自编码器的结构就是自编码器的一般结构，它使用多层感知机或者神经网络来充当编码器和解码器，神经网络的效果会好一些。变分自编码器产生的是原始图片的特征向量

## 5.2.Denoising AutoEncoders降噪自动编码器
降噪自动编码器实在自动编码器的基础上，训练数据假如噪声，所以自动编码器必须学习去除这种噪声而获得真正的没有被噪声污染过的输入，因此他的繁华能力比一般编码器强，它可以通过梯度下降法来训练

## 5.3.Sparse AutoEncoder稀疏自动编码器
在AutoEncoder的基础上记上L1正则化的限制(L1约束每一层节点中大部分都要为0，只有少数不为0，这也就是稀疏的名字来源)

# 6.Variational Autoencoder变分自编码器
![images](../images/02_04_002.png)

## 6.1.分布变换
VAE的目标是希望构建一个从隐变量$Z$生成目标数据$X$的模型。更准确地讲，VAE假设了服从某些常见的分布（比如正态分布或均匀分布），然后希望训练一个模型$X=g(Z)$，这个模型能够将原来的概率分布映射到训练集的概率分布，也就是说，它目的都是进行分布之间的变换。那现在假设$Z$服从标准的正态分布，那么我就可以从中采样得到若干个$Z_1,Z_2,…,Z_n$，然后对它做变换得到$X̂ 1=g(Z_1),X̂ 2=g(Z_2),…,X̂ n=g(Z_n)$，我们怎么判断这个通过f构造出来的数据集，它的分布跟我们目标的数据集分布是不是一样的呢？在生成对抗网络中，用的方法就是既然没有适合的度量，那就用神经网络来训练出来这个度量。在VAE中使用了一个精巧迂回的方法(由于我们现在只有样本本身，没有分布表达式，所以我们没办法使用KL散度来计算两个分布的相似度)

## 6.2.VAE介绍
首先我们有一批数据样本$\{x_1,x_2,...,x_n\}$，其整体用$X$来描述，我们想根据这些样本来得到$X$的分布表达式$p(X)$，如果能够得到的话，那直接根据$p(X)$来采样，就可以得到所有可能的$X$了(除了上述样本之外的)。当然这是一个终极理想的生成模型。但是这个模型很难实现，于是我们把分布改一改
$$p(X)=\sum_Zp(X|Z)p(Z)$$
这是根据概率的全概率公式的出来的。这里$p(X|Z)$表示一个由$Z$来生成$X$的模型，而我们假设$Z$服从标准正态分布，也就是$p(Z)=\mathcal{N}(0,I)$。如果这个理想能实现，那么我们就可以先从标准正态分布中采样一个Z，然后根据Z来算一个X，也是一个很棒的生成模型。<br/>
![images](../images/02_04_003.png)<br/>
这里有一个问题，因为你没办法确定生成出来的$z_k$是否还对应这原来的$x_k$，所以直接最小化$\mathcal{D}(\hat{X}_k,X_k)^2$是不对的。那该如何办呢？我们需要给每一个$x_k$都配上一个专属的正态分布$p(x_k|z_k)$，这样才科学，也方便后面的生成器做还原。但这样有多少个$X$就有多少个正态分布了。我们知道正态分布有两组参数$\mu$和$\sigma^2$，我们来用神经网络来拟合这两个参数

## 6.3.VAE神经网络
构建两个神经网络
$$\begin{eqnarray}
\mu_k&=&f_1(x_k)\\
\log\sigma^2&=&f_2(x_k)
\end{eqnarray}$$
我们选择拟合$log\sigma^2$而不是直接拟合$\sigma^2$，是因为$\sigma^2$总是非负的，需要加激活函数处理，而拟合$log\sigma^2$不需要加激活函数，因为它可正可负。到这里，我能知道专属于$x_k$的均值和方差了，也就知道它的正态分布长什么样了，然后从这个专属分布中采样一个$z_k$出来，然后经过一个生成器得到$\hat{x}_k=g(z_k)$，现在我们可以放心地最小化$\mathcal{D}(\hat{x}_k,x_k)^2$，因为$z_k$是从专属$x_k$的分布中采样出来的，这个生成器应该要把开始的$x_k$还原回来。于是可以画出VAE的示意图<br/>
![images](../images/02_04_004.png)<br/>

## 5.5.Adversarial Autoencoder对抗自编码器